# Beijing Air-Quality Time Series Project
### Modeling Time Series

by Dolci Sanders and Paul Torres



In [20]:
# Pandas/Data readers/ etc 
import pandas as pd
import numpy as np
import pickle

# Visuals 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Stats Models
from statsmodels.tsa.arima_model import ARMA,ARMAResults,ARIMA,ARIMAResults
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf # for determining (p,q) orders
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf # for determining (p,q) orders
from statsmodels.tsa.seasonal import seasonal_decompose      # for ETS Plots

# PMDARIMA
import pmdarima
from pmdarima import auto_arima    # for determining ARIMA orders

# Facebook Prophet 
from fbprophet import Prophet

# Metrics
from sklearn.metrics import mean_squared_error


import warnings
warnings.filterwarnings("ignore")

# Styling Guide 
plt.style.use('fivethirtyeight')
sns.set_palette(sns.diverging_palette(250,700, s= 70 , l= 10, n= 12))



## Read in pickle, check index for date time

In [21]:
train = pd.read_pickle('PKL/train.pkl')
test = pd.read_pickle('PKL/test.pkl')
time = pd.read_pickle('PKL/time.pkl')

In [26]:
train.index = pd.DatetimeIndex(train.index).to_period('D')
test.index = pd.DatetimeIndex(test.index).to_period('D')

In [27]:
train.head()

,PM2.5
Date,
2013-03-01,8.625000
2013-03-01,5.083333
2013-03-01,7.541667
2013-03-01,6.458333
2013-03-01,7.541667


## Find Ideal Parameters

In [29]:
from pmdarima.arima.utils import ndiffs

In [30]:
## Adf Test
print(ndiffs(train, test='adf')) 
# KPSS test
print(ndiffs(train, test='kpss'))
# PP test:
print(ndiffs(train, test='pp'))

0
0
0


In [31]:
stepwise_fit = auto_arima(train['PM2.5'], start_p=0, start_q=0,
                         max_p=2, max_q=2, m=12,
                         seasonal=True,
                         d=None, trace=True,
                         error_action='ignore',   # we don't want to know if an order does not work
                         suppress_warnings=True,  # we don't want convergence warnings
                         stepwise=True)           # set to stepwise

stepwise_fit.summary()

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(1,0,1)[12] intercept   : AIC=147356.253, Time=38.33 sec
 ARIMA(0,0,0)(0,0,0)[12] intercept   : AIC=152248.397, Time=0.49 sec
 ARIMA(1,0,0)(1,0,0)[12] intercept   : AIC=129840.070, Time=17.63 sec
 ARIMA(0,0,1)(0,0,1)[12] intercept   : AIC=140073.046, Time=19.16 sec
 ARIMA(0,0,0)(0,0,0)[12]             : AIC=163790.563, Time=0.19 sec
 ARIMA(1,0,0)(0,0,0)[12] intercept   : AIC=129846.276, Time=0.91 sec
 ARIMA(1,0,0)(2,0,0)[12] intercept   : AIC=129772.286, Time=68.28 sec
 ARIMA(1,0,0)(2,0,1)[12] intercept   : AIC=129749.209, Time=115.53 sec
 ARIMA(1,0,0)(1,0,1)[12] intercept   : AIC=129822.028, Time=42.77 sec
 ARIMA(1,0,0)(2,0,2)[12] intercept   : AIC=129749.643, Time=95.22 sec
 ARIMA(1,0,0)(1,0,2)[12] intercept   : AIC=129752.703, Time=96.09 sec
 ARIMA(0,0,0)(2,0,1)[12] intercept   : AIC=147513.434, Time=113.78 sec
 ARIMA(2,0,0)(2,0,1)[12] intercept   : AIC=128441.938, Time=156.86 sec
 ARIMA(2,0,0)(1,0,1)[12] intercept   : AIC=1286

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                                  y   No. Observations:                13452
Model:             SARIMAX(1, 0, 1)x(2, 0, 1, 12)   Log Likelihood              -64086.131
Date:                            Thu, 10 Sep 2020   AIC                         128186.261
Time:                                    17:09:39   BIC                         128238.809
Sample:                                         0   HQIC                        128203.789
                                          - 13452                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      1.4648      0.181      8.085      0.000       1.110       1.820
ar.L1          0.9688      0.002    568.519      0.000       0.965       0.972
ma.L1         -0.3841      0.004    -97.950      0.000      -0.392      -0.376
ar.S.L12       0.5329      0.028     19.367      0.000       0.479       0.587
ar.S.L24      -0.1148      0.005    -25.346      0.000      -0.124      -0.106
ma.S.L12      -0.5422      0.027    -19.727      0.000      -0.596      -0.488
sigma2       804.9468      3.363    239.333      0.000     798.355     811.539
===================================================================================
Ljung-Box (Q):                      221.51   Jarque-Bera (JB):            264483.79
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               1.01   Skew:                            -0.23
Prob(H) (two-sided):                  0.70   Kurtosis:                        24.72
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

### Above the best model:  ARIMA(1,0,1)(2,0,1)[12] intercept


## Seasonal Arima

In [32]:
auto_arima(train['PM2.5'],seasonal=True).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                13452
Model:               SARIMAX(3, 0, 0)   Log Likelihood              -64190.009
Date:                Thu, 10 Sep 2020   AIC                         128390.017
Time:                        17:11:17   BIC                         128427.552
Sample:                             0   HQIC                        128402.537
                              - 13452                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      4.9253      0.501      9.837      0.000       3.944       5.907
ar.L1          0.5961      0.004    157.025      0.000       0.589       0.604
ar.L2          0.2098      0.005     41.930      0.000       0.200       0.220
ar.L3          0.1331      0.005     29.031      0.000       0.124       0.142
sigma2       817.0220      3.539    230.874      0.000     810.086     823.958
===================================================================================
Ljung-Box (Q):                      382.28   Jarque-Bera (JB):            267772.09
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               1.00   Skew:                            -0.39
Prob(H) (two-sided):                  0.93   Kurtosis:                        24.84
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

## Sarimax Model

SARIMAX(2,2,0) MSE Error:  9042.593602878254
SARIMAX(2,2,0) RMSE Error:  95.09255282554074

In [41]:
model_SARIMAX = SARIMAX(train['PM2.5'],order=(0,0,1))
results = model_SARIMAX.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                  PM2.5   No. Observations:                13452
Model:               SARIMAX(0, 0, 1)   Log Likelihood              -75609.778
Date:                Thu, 10 Sep 2020   AIC                         151223.557
Time:                        17:23:43   BIC                         151238.571
Sample:                    03-01-2013   HQIC                        151228.565
                         - 03-25-2016                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.7803      0.004    220.359      0.000       0.773       0.787
sigma2      4462.5559     35.028    127.399      0.000    4393.902    4531.210
===================================================================================
Ljung-Box (Q):                    44045.98   Jarque-Bera (JB):             26032.69
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.86   Skew:                             1.70
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.90
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [42]:
#Obtain predicted values

start=len(train)
end=len(train)+len(test)-1
predictions_SARIMAX = results.predict(start=start, end=end, dynamic=False).rename('SARIMAX(0,0,1) Predictions')

In [43]:
for i in range(len(predictions_SARIMAX)):
    print(f"predicted={predictions_SARIMAX[i]:<19}, expected={test['PM2.5'][i]}")


predicted=16.80228676181178  , expected=13.833333333333334
predicted=0.0                , expected=22.166666666666668
predicted=0.0                , expected=12.208333333333334
predicted=0.0                , expected=12.583333333333334
predicted=0.0                , expected=10.75
predicted=0.0                , expected=9.208333333333334
predicted=0.0                , expected=12.125
predicted=0.0                , expected=8.565217391304348
predicted=0.0                , expected=8.541666666666666
predicted=0.0                , expected=12.782608695652174
predicted=0.0                , expected=10.75
predicted=0.0                , expected=12.375
predicted=0.0                , expected=15.833333333333334
predicted=0.0                , expected=12.521739130434783
predicted=0.0                , expected=16.75
predicted=0.0                , expected=15.208333333333334
predicted=0.0                , expected=17.956521739130434
predicted=0.0                , expected=16.083333333333332
pred

In [44]:
from sklearn.metrics import mean_squared_error
error = mean_squared_error(test['PM2.5'], predictions_SARIMAX)
print(f'SARIMAX(2,2,0) MSE Error: {error:18}')

from statsmodels.tools.eval_measures import rmse
error = rmse(test['PM2.5'], predictions_SARIMAX)
print(f'SARIMAX(2,2,0) RMSE Error: {error:18}')


SARIMAX(2,2,0) MSE Error:  9042.593602878254
SARIMAX(2,2,0) RMSE Error:  95.09255282554074
